In [1]:
import sys
#from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import from_unixtime, col, to_timestamp, split, explode, sum, count, row_number, desc
from pyspark.sql.types import StructType,TimestampType, StringType, IntegerType, DoubleType
from datetime import datetime

In [2]:
# Create spark session
def init_spark():
  spark = (SparkSession
    .builder
    .getOrCreate())
  sc = spark.sparkContext
  return spark,sc

In [3]:
def getLogger(spark):
    log4j_logger = spark._jvm.org.apache.log4j
    logger = log4j_logger.LogManager.getRootLogger()
    return logger

In [4]:
spark,sc = init_spark()
#logger = getLogger(spark)
#sys.stdout = open(sys.stdout.fileno(), mode='w', encoding='utf8', buffering=1)

23/09/15 09:06:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
loanDataDynamic ="Loan_Book_Dynamic.csv" #contains loan information
loanDataSpacr = "Loan_Book_Stacr.csv" #contains loan in current state and the flood risk data
floodData = "FloridaPropertyLocationAndFlood_v1_0.csv" # flood risk data associated with the loan id
dtStr = datetime.today().strftime('%Y%m%d')

In [6]:
#logger.info("######################################")
#logger.info("READING INPUT FILES")
#logger.debug("Loan Data SPACR :: "+loanDataDynamic)
##logger.debug("Loan Data Dynamic :: "+loanDataSpacr)
#logger.debug("Flood Index :: "+floodData)
#logger.debug("Date :: "+dtStr)
#logger.info("######################################")

In [7]:
####################################
# Read CSV Data
####################################
#logger.info("######################################")
#logger.info("READING ECAD CSV DATA ")
#logger.info(loanDataDynamic)
#logger.info("######################################")

In [8]:
loandata_dynamic_schema = StructType().add("ASOFMON",StringType(),True) .add("LOAN_ID",StringType(),True) .add("SERVICER_NAME",StringType(),True) .add("LOAN_AGE",IntegerType(),True) .add("REM_MONTHS_LEGAL_MATURITY",StringType(),True) .add("ADJ_REM_MONTHS_MATURITY",IntegerType(),True) .add("CUR_DELINQUENCY_STATUS",StringType(),True) .add("PAYMENT_HISTORY",StringType(),True) .add("CUR_INT_RATE",DoubleType(),True) .add("CUR_ACTUAL_UPB",DoubleType(),True) .add("CUR_INT_BEARING_UPB",DoubleType(),True) .add("UPB_AT_REMOVAL",DoubleType(),True) .add("ZERO_BAL_CODE",IntegerType(),True) .add("ZERO_BAL_DATE",StringType(),True) .add("UNDERWRITING_DEFECT_SETTLEMENT_DATE",StringType(),True) .add("MOD_FLAG",StringType(),True) .add("DISASTER_FORBEARANCE_STATUS",IntegerType(),True) .add("DUE_DATE_LAST_P_I",StringType(),True) .add("BANKUPTACY_FLAG",StringType(),True) .add("FORECLOSURE_DATE",StringType(),True) .add("NET_SALES_PROCEEDS",DoubleType(),True) .add("MI_CREDIT",DoubleType(),True) .add("TAXES_AND_INSURANCE",DoubleType(),True) .add("LEGAL_COST",DoubleType(),True) .add("MAINTENANCE_PRESERVATION_COST",DoubleType(),True) .add("BANKRUPTCY_CRAMDOWN_COST",IntegerType(),True) .add("MISC_EXPENSES",DoubleType(),True) .add("MISC_CREDITS",DoubleType(),True) .add("MI_CANCELLATION_INDICATOR",StringType(),True) .add("ESTIMATED_LTV",IntegerType(),True) .add("FORECAST_STANDARD_DEVIATION",StringType(),True) .add("CREDIT_SCORE_CURRENT_FICO",IntegerType(),True) .add("UPDATED_CREDIT_SCORE_2",IntegerType(),True) .add("MOD_NO",IntegerType(),True) .add("MOD_PROGRAM",StringType(),True) .add("MOD_TYPE",StringType(),True) .add("MOD_FIRST_PAY_DATE",StringType(),True) .add("MOD_DTI_RATIO",IntegerType(),True) .add("TOTAL_CAPITALIZED_AMT",DoubleType(),True) .add("INTEREST_RATE_INDICATOR",StringType(),True) .add("FIRST_RATE_ADJ_DATE",StringType(),True) .add("FIRST_RATE",StringType(),True) .add("SCOND_RATE_ADJ_DATE",StringType(),True) .add("SCOND_RATE",StringType(),True) .add("THIRD_RATE_ADJ_DATE",StringType(),True) .add("THIRD_RATE",StringType(),True) .add("FOURTH_RATE_ADJ_DATE",StringType(),True) .add("FOURTH_RATE",StringType(),True) .add("FIFTH_RATE_ADJ_DATE",StringType(),True) .add("FIFTH_RATE",StringType(),True) .add("DELINQUENCY_ACCRUED_INT",DoubleType(),True) .add("MODIFICATION_COST",DoubleType(),True) .add("LOAN_IN_ELIGIBLE_DISASTER_AREA",StringType(),True) .add("UPDATED_CREDIT_SCORE_3",StringType(),True) .add("ALTERNATIVE_APPRAISAL_TYPE",StringType(),True) .add("GROUP_NUMBER",IntegerType(),True) .add("ENHANCED_RELIEF_REFI_INDICATOR",IntegerType(),True) .add("BORROWER_ASSISTANCE_PLAN",IntegerType(),True) .add("PAYMENT_DEFERRAL_FLAG",StringType(),True) .add("DISTRESSED_PRINCIPAL_BALANCE_FLAG",StringType(),True)

In [9]:
df_loandata_dynamic_csv = (
        spark.read
        .format("csv")
        .option("header", True)
        .option("encoding", "UTF-8")
        .option("mode", "PERMISSIVE")
        .option("columnNameOfCorruptRecord", "_corrupt_record")
        .schema(loandata_dynamic_schema)
        .load(loanDataDynamic)
    )

In [10]:
df_loandata_dynamic_csv.printSchema

<bound method DataFrame.printSchema of DataFrame[ASOFMON: string, LOAN_ID: string, SERVICER_NAME: string, LOAN_AGE: int, REM_MONTHS_LEGAL_MATURITY: string, ADJ_REM_MONTHS_MATURITY: int, CUR_DELINQUENCY_STATUS: string, PAYMENT_HISTORY: string, CUR_INT_RATE: double, CUR_ACTUAL_UPB: double, CUR_INT_BEARING_UPB: double, UPB_AT_REMOVAL: double, ZERO_BAL_CODE: int, ZERO_BAL_DATE: string, UNDERWRITING_DEFECT_SETTLEMENT_DATE: string, MOD_FLAG: string, DISASTER_FORBEARANCE_STATUS: int, DUE_DATE_LAST_P_I: string, BANKUPTACY_FLAG: string, FORECLOSURE_DATE: string, NET_SALES_PROCEEDS: double, MI_CREDIT: double, TAXES_AND_INSURANCE: double, LEGAL_COST: double, MAINTENANCE_PRESERVATION_COST: double, BANKRUPTCY_CRAMDOWN_COST: int, MISC_EXPENSES: double, MISC_CREDITS: double, MI_CANCELLATION_INDICATOR: string, ESTIMATED_LTV: int, FORECAST_STANDARD_DEVIATION: string, CREDIT_SCORE_CURRENT_FICO: int, UPDATED_CREDIT_SCORE_2: int, MOD_NO: int, MOD_PROGRAM: string, MOD_TYPE: string, MOD_FIRST_PAY_DATE: stri

In [11]:
df_loandata_dynamic_csv.show(10,False)

23/09/15 09:06:50 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------+------------+-----------------------------------------+--------+-------------------------+-----------------------+----------------------+---------------+------------+--------------+-------------------+--------------+-------------+-------------+-----------------------------------+--------+---------------------------+-----------------+---------------+----------------+------------------+---------+-------------------+----------+-----------------------------+------------------------+-------------+------------+-------------------------+-------------+---------------------------+-------------------------+----------------------+------+-----------+--------+------------------+-------------+---------------------+-----------------------+-------------------+----------+-------------------+----------+-------------------+----------+--------------------+-----------+-------------------+----------+-----------------------+-----------------+------------------------------+----------------------+-

In [12]:
loandata_spacr_schema = StructType().add("ASOFMON",StringType(),True) .add("LOAN_ID",StringType(),True) .add("PRODUCT_TYPE",StringType(),True) .add("SELLER_NAME",StringType(),True) .add("PROPERTY_STATE",StringType(),True) .add("MSA",IntegerType(),True) .add("FIRST_PAY_DATE",StringType(),True) .add("MATURITY_DATE",StringType(),True) .add("ORIG_LOAN_TERM",IntegerType(),True) .add("ORIG_INT_RATE",DoubleType(),True) .add("ORIG_UPB",IntegerType(),True) .add("UPB_ISSUANCE",DoubleType(),True) .add("LOAN_PURPOSE",StringType(),True) .add("CHANNEL",StringType(),True) .add("PROPERTY_TYPE",StringType(),True) .add("NUMBER_OF_UNITS",IntegerType(),True) .add("OCCUPANCY_STATUS",StringType(),True) .add("NUMBER_OF_BORROWERS",IntegerType(),True) .add("FIRST_BUYER_INDICATOR",StringType(),True) .add("PRE_PAYMENT_PENALTY_INDICATOR",StringType(),True) .add("CREDIT_SCORE_ORIGINATION",IntegerType(),True) .add("ORIG_LTV",IntegerType(),True) .add("ORIG_CLTV",IntegerType(),True) .add("ORIG_DTI",IntegerType(),True) .add("MI_PCT",IntegerType(),True) .add("ESTIMATED_LTV",StringType(),True) .add("BORROWER_PAID",StringType(),True) .add("PROGRAM_INDICATOR",StringType(),True) .add("MTG_INSURANCE_TYPE",IntegerType(),True) .add("ALTERNATIVE_APPRAISAL_TYPE",StringType(),True) .add("GROUP_NUMBER",IntegerType(),True) .add("ENHANCED_RELIEF_REFI_INDICATOR",IntegerType(),True) .add("PROPERTY_ZIP",IntegerType(),True)
df_loandata_spacr_csv = (
        spark.read
        .format("csv")
        .option("header", True)
        .option("encoding", "UTF-8")
        .option("mode", "PERMISSIVE")
        .option("columnNameOfCorruptRecord", "_corrupt_record")
        .schema(loandata_spacr_schema)
        .load(loanDataSpacr)
    )

In [13]:
df_loandata_spacr_csv.printSchema

<bound method DataFrame.printSchema of DataFrame[ASOFMON: string, LOAN_ID: string, PRODUCT_TYPE: string, SELLER_NAME: string, PROPERTY_STATE: string, MSA: int, FIRST_PAY_DATE: string, MATURITY_DATE: string, ORIG_LOAN_TERM: int, ORIG_INT_RATE: double, ORIG_UPB: int, UPB_ISSUANCE: double, LOAN_PURPOSE: string, CHANNEL: string, PROPERTY_TYPE: string, NUMBER_OF_UNITS: int, OCCUPANCY_STATUS: string, NUMBER_OF_BORROWERS: int, FIRST_BUYER_INDICATOR: string, PRE_PAYMENT_PENALTY_INDICATOR: string, CREDIT_SCORE_ORIGINATION: int, ORIG_LTV: int, ORIG_CLTV: int, ORIG_DTI: int, MI_PCT: int, ESTIMATED_LTV: string, BORROWER_PAID: string, PROGRAM_INDICATOR: string, MTG_INSURANCE_TYPE: int, ALTERNATIVE_APPRAISAL_TYPE: string, GROUP_NUMBER: int, ENHANCED_RELIEF_REFI_INDICATOR: int, PROPERTY_ZIP: int]>

In [14]:
df_loandata_spacr_csv.show(10,False)

+----------+------------+------------+------------------------------+--------------+-----+--------------+-------------+--------------+-------------+--------+------------+------------+-------+-------------+---------------+----------------+-------------------+---------------------+-----------------------------+------------------------+--------+---------+--------+------+-------------+-------------+-----------------+------------------+--------------------------+------------+------------------------------+------------+
|ASOFMON   |LOAN_ID     |PRODUCT_TYPE|SELLER_NAME                   |PROPERTY_STATE|MSA  |FIRST_PAY_DATE|MATURITY_DATE|ORIG_LOAN_TERM|ORIG_INT_RATE|ORIG_UPB|UPB_ISSUANCE|LOAN_PURPOSE|CHANNEL|PROPERTY_TYPE|NUMBER_OF_UNITS|OCCUPANCY_STATUS|NUMBER_OF_BORROWERS|FIRST_BUYER_INDICATOR|PRE_PAYMENT_PENALTY_INDICATOR|CREDIT_SCORE_ORIGINATION|ORIG_LTV|ORIG_CLTV|ORIG_DTI|MI_PCT|ESTIMATED_LTV|BORROWER_PAID|PROGRAM_INDICATOR|MTG_INSURANCE_TYPE|ALTERNATIVE_APPRAISAL_TYPE|GROUP_NUMBER|ENHAN

In [15]:
floodDataSchema = StructType().add("ID",StringType(),True) .add("LOAN_ID",StringType(),True) .add("LATITUDE",DoubleType(),True) .add("LONGITUDE",DoubleType(),True) .add("River_Floodscore_Def",IntegerType(),True) .add("River_Floodscore_UD",IntegerType(),True) .add("Coastal_Floodscore_UD",IntegerType(),True) .add("Surfacewater_Floodscore_UD",IntegerType(),True) .add("FloodScore_Def",IntegerType(),True) .add("FloodScore_UD",IntegerType(),True) .add("River_Floodscore_Def_RCP45_2050",IntegerType(),True) .add("River_Floodscore_UD_RCP45_2050",IntegerType(),True) .add("Surfacewater_Floodscore_UD_RCP45_2050",IntegerType(),True) .add("River_Floodscore_Def_RCP85_2050",IntegerType(),True) .add("River_Floodscore_UD_RCP85_2050",IntegerType(),True) .add("Surfacewater_Floodscore_UD_RCP85_2050",IntegerType(),True)
floodData_csv = (
        spark.read
        .format("csv")
        .option("header", True)
        .option("encoding", "UTF-8")
        #.option("mode", "PERMISSIVE")
        #.option("columnNameOfCorruptRecord", "_corrupt_record")
        .option("inferSchema",True)
        #.schema(floodDataSchema)
        .load(floodData)
    )



In [16]:
floodData_csv.printSchema

<bound method DataFrame.printSchema of DataFrame[ID: string, LOAN_ID: string, LATITUDE: double, LONGITUDE: double, River_Floodscore_Def: int, River_Floodscore_UD: int, Coastal_Floodscore_UD: int, Surfacewater_Floodscore_UD: int, FloodScore_Def: int, FloodScore_UD: int, River_Floodscore_Def_RCP45_2050: int, River_Floodscore_UD_RCP45_2050: int, Surfacewater_Floodscore_UD_RCP45_2050: int, River_Floodscore_Def_RCP85_2050: int, River_Floodscore_UD_RCP85_2050: int, Surfacewater_Floodscore_UD_RCP85_2050: int]>

In [17]:
floodData_csv.show(10,False)

+----------+------------+-----------+------------+--------------------+-------------------+---------------------+--------------------------+--------------+-------------+-------------------------------+------------------------------+-------------------------------------+-------------------------------+------------------------------+-------------------------------------+
|ID        |LOAN_ID     |LATITUDE   |LONGITUDE   |River_Floodscore_Def|River_Floodscore_UD|Coastal_Floodscore_UD|Surfacewater_Floodscore_UD|FloodScore_Def|FloodScore_UD|River_Floodscore_Def_RCP45_2050|River_Floodscore_UD_RCP45_2050|Surfacewater_Floodscore_UD_RCP45_2050|River_Floodscore_Def_RCP85_2050|River_Floodscore_UD_RCP85_2050|Surfacewater_Floodscore_UD_RCP85_2050|
+----------+------------+-----------+------------+--------------------+-------------------+---------------------+--------------------------+--------------+-------------+-------------------------------+------------------------------+------------------------

In [18]:
#data profiling for loan data
for col in df_loandata_dynamic_csv.columns:
    df_loandata_dynamic_csv = df_loandata_dynamic_csv.withColumnRenamed(col, col.lower())

df_loandata_dynamic_csv.groupBy("loan_id").agg(count("loan_id").alias("loan_id_count")).orderBy(desc("loan_id_count")).show(100,False)
    

+------------+-------------+
|loan_id     |loan_id_count|
+------------+-------------+
|13DN01064831|1            |
|13DN02011939|1            |
|13DN02061197|1            |
|13DN02092597|1            |
|13DN02112267|1            |
|13DN02119730|1            |
|13DN02126363|1            |
|13DN02135223|1            |
|14DN01035122|1            |
|14DN01058837|1            |
|14DN01066545|1            |
|14DN02054891|1            |
|14DN02061624|1            |
|14DN02063518|1            |
|14DN02088202|1            |
|14DN03001296|1            |
|14DN03003235|1            |
|14DN03004897|1            |
|14DN03007832|1            |
|14DN03065568|1            |
|14DN03066405|1            |
|14DN04001786|1            |
|14DN04032244|1            |
|14DN04051685|1            |
|14HQ02014714|1            |
|14HQ02048674|1            |
|14HQ02084136|1            |
|15DN01009023|1            |
|15DN01010755|1            |
|15DN01020506|1            |
|15DN01038071|1            |
|15DN01044706|

In [19]:
for col in df_loandata_spacr_csv.columns:
    df_loandata_spacr_csv = df_loandata_spacr_csv.withColumnRenamed(col, col.lower())
    
df_loandata_spacr_csv.groupBy("loan_id").agg(count("loan_id").alias("loan_id_count")).orderBy(desc("loan_id_count")).show(100,False)

+------------+-------------+
|loan_id     |loan_id_count|
+------------+-------------+
|13DN01008599|1            |
|13DN01023014|1            |
|13DN01029881|1            |
|13DN01065837|1            |
|13DN02000426|1            |
|13DN02101931|1            |
|13DN02110336|1            |
|13DN02124925|1            |
|14DN01038041|1            |
|14DN01078495|1            |
|14DN01078604|1            |
|14DN01083641|1            |
|14DN01124902|1            |
|14DN02051103|1            |
|14DN02068352|1            |
|14DN02069196|1            |
|14DN02104512|1            |
|14DN03031686|1            |
|14DN03032649|1            |
|14DN03041052|1            |
|14DN03046928|1            |
|14DN04000050|1            |
|14DN04036310|1            |
|14HQ02010755|1            |
|14HQ02143727|1            |
|15DN01029339|1            |
|15DN01039892|1            |
|15DN01039903|1            |
|15DN01098299|1            |
|15DN01115667|1            |
|15DNA1015627|1            |
|15DNA1034376|

In [20]:
for col in floodData_csv.columns:
    floodData_csv = floodData_csv.withColumnRenamed(col, col.lower())
    
floodData_csv.groupBy("loan_id").agg(count("loan_id").alias("loan_id_count")).orderBy(desc("loan_id_count")).show(100,False)

+------------+-------------+
|loan_id     |loan_id_count|
+------------+-------------+
|15DN01114915|1            |
|20HQA5049373|1            |
|21HQA1057323|1            |
|22DNA3010835|1            |
|22DNA7064733|1            |
|18HQA2155900|1            |
|19FTR4102023|1            |
|18DNA2114017|1            |
|16DNA3068236|1            |
|20HQA1015103|1            |
|20HQA1061076|1            |
|16DNA4025253|1            |
|21DNA1021428|1            |
|21DNA1025686|1            |
|21DNA1028190|1            |
|21DNA7195981|1            |
|20DNA1049259|1            |
|15HQA2035744|1            |
|21DNA7225784|1            |
|20DNA3142148|1            |
|21HQA1020818|1            |
|18DNA2025934|1            |
|20DNA3163149|1            |
|20DNA4045380|1            |
|16DNA1015013|1            |
|19FTR1028442|1            |
|18DNA2152085|1            |
|18DNA2156102|1            |
|18DNA2158537|1            |
|18HRP1091090|1            |
|22DNA1087601|1            |
|20DNA6042514|

In [21]:
df_loandata_dynamic_cols = df_loandata_dynamic_csv.columns
df_loandata_spacr_cols = df_loandata_spacr_csv.columns
floodData_cols = floodData_csv.columns

#check if loan data spacr cols exist in loan data dynamic

for i in df_loandata_spacr_cols:
    if i in df_loandata_dynamic_cols : print("df_dynamic contains "+i)
        

df_dynamic contains asofmon
df_dynamic contains loan_id
df_dynamic contains estimated_ltv
df_dynamic contains alternative_appraisal_type
df_dynamic contains group_number
df_dynamic contains enhanced_relief_refi_indicator


In [24]:
#create unified loan data with flood rating
df_unified = df_loandata_dynamic_csv.join(df_loandata_spacr_csv, df_loandata_dynamic_csv.loan_id == df_loandata_spacr_csv.loan_id,"left_outer")
#drop duplicate columns
duplicate_cols = [c for c in df_loandata_spacr_csv.columns if c in df_loandata_dynamic_csv.columns]
for col in duplicate_cols:
    df_unified = df_unified.drop(df_loandata_spacr_csv[col])
    
df_unified.printSchema

<bound method DataFrame.printSchema of DataFrame[asofmon: string, loan_id: string, servicer_name: string, loan_age: int, rem_months_legal_maturity: string, adj_rem_months_maturity: int, cur_delinquency_status: string, payment_history: string, cur_int_rate: double, cur_actual_upb: double, cur_int_bearing_upb: double, upb_at_removal: double, zero_bal_code: int, zero_bal_date: string, underwriting_defect_settlement_date: string, mod_flag: string, disaster_forbearance_status: int, due_date_last_p_i: string, bankuptacy_flag: string, foreclosure_date: string, net_sales_proceeds: double, mi_credit: double, taxes_and_insurance: double, legal_cost: double, maintenance_preservation_cost: double, bankruptcy_cramdown_cost: int, misc_expenses: double, misc_credits: double, mi_cancellation_indicator: string, estimated_ltv: int, forecast_standard_deviation: string, credit_score_current_fico: int, updated_credit_score_2: int, mod_no: int, mod_program: string, mod_type: string, mod_first_pay_date: stri

In [25]:
df_unified.coalesce(1).write.json("vichara_loan_flood_data.json")
df_unified.coalesce(1).write.options(header='True', delimiter=',').csv("vichara_loan_flood_data.csv")